In [ ]:
import pandas as pd
import numpy as np
import neptune
import random
import gc
import ast

from lightgbm import LGBMClassifier


from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.base import clone
from sklearn.model_selection import KFold

In [ ]:
## инициализируем проект в нептуне
neptune.init('iliaavilov/Zindi-insurance')

In [ ]:
random_state = 555

# Загрузка данных

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [ ]:
def encoding(data_train, data_test, column):
    
    """
    Кодировка колонки датафрейма через LabelEncoder из sklearn

    ----------
    data_train - тренировочный датафрейм
    data_test - тестовый датафрейм
    column - колонка датафрейма, которую надо закодировать
    ----------

    """
    
    le = LabelEncoder()
    le.fit(np.append(data_train[column].values, data_test[column].values))
    data_train[column] = le.transform(data_train[column].values)
    data_test[column] = le.transform(data_test[column].values)
    return data_train, data_test

# Информация о продуктах, которые уже есть у пользователя

In [ ]:
## Выбираем продукты, которые уже есть у пользователей
real_1 = test[test['presence'] == 1][['ID', 'variable']]
real_1['Label'] = 1


## Оставляем в тестовом датасете только продукты, информации о наличии которых нет
test = test[test['presence'] == 0]
ID_X_var = test[['ID', 'variable']]

In [ ]:
train, test = encoding(train, test, 'variable')

# Разбиваем на X и y

In [ ]:
X_train = train.drop(['presence', 'ID'], axis = 'columns')
y_train = train['presence']
test = test.drop(['presence', 'ID'], axis = 'columns')
del train
gc.collect()

# Делаем предсказания

In [ ]:
def predicting(X_train, y_train, X_test,  model, ID_X_var):
    
    """
    Предсказывает target с помощью модели с заданными параметрами и автоматически сохраняет csv файл с предсказаниями

    ----------
    X_train - тренировочные данные
    y_train - тренировочный target
    X_test - тестовые данные
    model - модель с заданными параметрами
    ID_X_Var - датафрейм с ID пользователя и товара
    ----------

    """
    
    model.fit(X_train, y_train)
    probas = model.predict_proba(X_test).T[1]
    
    submission = pd.DataFrame({
        'ID X PCODE' : ID_X_var['ID'].values + ' X ' + ID_X_var['variable'].values,
        'Label': probas})
    submission = submission.append(pd.DataFrame(
        {
        'ID X PCODE' : real_1['ID'].values + ' X ' + real_1['variable'].values,
        'Label': real_1['Label']
        }))
    
    submission.reset_index(drop = True, 
                           inplace = True)
    submission.to_csv('submission.csv', 
                      index = False)
    
    return(submission)

In [ ]:
base_params =  {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'n_jobs': -1,
    'n_estimators': 1500,
    'random_state': random_state,
    'categorical_feature': [X_train.columns.get_loc(cat_col) for cat_col in 
                            ['sex', 'marital_status', 'branch_code', 'occupation_code',
                             'occupation_category_code', 'variable', 'P5DA',
                             'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9',
                             'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW',
                             'GHYX', 'ECY3']]}


## Выбираем параметры, которые дали наилучший результат на валидации
best_params = ast.literal_eval(neptune.project.get_experiments(""id эксперимента"")[0].get_properties()['best_parameters'])
parameters = {**base_params, **best_params}

In [ ]:
predicting(X_train, y_train, test,  LGBMClassifier(**parameters), ID_X_var, real_1)

In [ ]:
# Логируем информацию о результате на лидерборде
neptune.project.get_experiments(""id эксперимента"")[0].log_metric('test_score', )